In [4]:
import matplotlib.pyplot as plt

import re

import pandas as pd

import numpy as np

pd.options.display.max_columns = 30

from numpy import *

import plotly.express as px

import plotly.graph_objects as go

import seaborn as sns

from wordcloud import WordCloud, STOPWORDS

from sklearn import *


                                           Open Documents and save contents as string variables

In [5]:
EssayFileName = "Vaccines are Beneficial.txt"

ResumeFileName = "Resume.txt"

LabFileName = "EEE 334 Lab 1.txt"


EssayLines = ""

Resumelines = ""

LabLines = ""


def Read(FileName, lines):
    with open(FileName, "r", encoding = 'utf-8') as File:
        lines = File.read()
    return(lines)
    
EssayText = Read(EssayFileName, EssayLines)

ResumeText = Read(ResumeFileName, Resumelines)

LabText = Read(LabFileName, LabLines)

    

                                                Clean  and Sort Data

In [6]:
def CleanData(Text):
    Text = re.sub(r'[^a-zA-Z]', '', Text)
    
    Text = Text.lower()
    
    return(Text)

EssayText = CleanData(EssayText)

ResumeText = CleanData(ResumeText)

LabText = CleanData(LabText) 

I removed all non-alphabetic characters and changed all uppercase letters to lowercase.  That ensures I can obtain the frequency of just the letters and that all instances of the same letter will counted without regard to case.

                                                 Get Letter Frequencies as Dictionaries 

In [7]:
def GetFrequencies(Text):  
    LetterFrequencies = {}
    for keys in Text:
        LetterFrequencies[keys] = LetterFrequencies.get(keys, 0) + 1
    return(LetterFrequencies)

EssayFrequencyDict = GetFrequencies(EssayText)

ResumeFrequencyDict = GetFrequencies(ResumeText)

LabFrequencyDict = GetFrequencies(LabText)

                                                Convert Dictionaries to Data Frames and Sort Frequency by Letter

In [8]:
def ConvertAndSort(FrequencyDict, column2):
    Frequency_df = pd.DataFrame(FrequencyDict.items(), columns=['Letter', column2]) 

    Frequency_df = Frequency_df.sort_values("Letter")

    Frequency_df = Frequency_df[Frequency_df.Letter != " "].reset_index()  #renumber rows from 0 to 1 from the top down
    
    Frequency_df.pop('index')  #remove column containing prior order as I have no need of it
    
    Frequency_df.dropna(inplace = True) 
    
    return(Frequency_df)
    
EssayFrequency_df = ConvertAndSort(EssayFrequencyDict, 'Essay Frequency')

ResumeFrequency_df = ConvertAndSort(ResumeFrequencyDict, 'Resume Frequency')

LabFrequency_df = ConvertAndSort(LabFrequencyDict, 'Lab Frequency')


                                                       Merge Dataframes and Get Proportions

In [39]:
#"Letter" column in each df is the same, so two of them are being concantenated without it
temp = pd.concat([EssayFrequency_df, ResumeFrequency_df.drop(['Letter'], axis = 1, inplace = False)], axis = 1, join = 'outer')

merged_data = pd.concat([temp, LabFrequency_df.drop(['Letter'], axis = 1, inplace = False)], axis = 1, join = 'outer')

merged_data.dropna(inplace = True)

merged_data.columns = ["Letter", "Essay", "Resume", "Lab"]

frequency_proportions = merged_data.copy()

temp = frequency_proportions.iloc[:, 1:4]

frequency_proportions.iloc[:, 1:4] = temp/sum(temp)

frequency_proportions.columns = ["Letter", "Essay", "Resume", "Lab"]

                                                         Get Summary Stats

Since the data was sorted by letter in the previous step, each frequency category doesn't need its own separate letter category, hence, I dropped that column from the resume and lab df's, keeping it on the essay df, and then concantenate them.

In [40]:
SummaryStats = (merged_data.describe()).round(decimals = 2)

SummaryStats_Proportions = (frequency_proportions.describe()).round(decimals = 3)

summary_stats = pd.concat([SummaryStats, SummaryStats_Proportions], axis = 1, join = 'outer', 
                           keys = ['Letter Counts', ' Letter Proportions'])

summary_stats.drop('count', inplace = True)

summary_stats

Letter Counts                   Letter Proportions              
             Essay  Resume      Lab               Essay Resume    Lab
mean        530.85  110.31   479.58               0.038  0.038  0.038
std         477.23   95.49   392.42               0.035  0.033  0.031
min           7.00    5.00    14.00               0.001  0.002  0.001
25%         189.00   27.25   197.00               0.014  0.010  0.016
50%         332.00   93.00   375.00               0.024  0.032  0.030
75%         864.25  185.00   810.75               0.063  0.065  0.065
max        1901.00  300.00  1471.00               0.138  0.105  0.118

                                                        Summary Stats by Letter

In [ ]:
#easiest way to get stats for data grouped by letter is to make the letters index names and transpose the df.  
#When I used the groupby function instead, .describe gave weird results.

make_letters_index = merged_data.copy() 
#I don't want to make the letters variables on the original df though because still I want 
#to be able to use 'Letter' as a grouping variable for graphing.  Hence, the use of the copy function 
#so that changes to make_letters_index aren't seen by merged_data

make_letters_index.index = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 
                     'h', 'i', 'j', 'k', 'l', 'm', 'n', 
                     'o', 'p', 'q', 'r', 's', 't', 'u', 
                     'v', 'w', 'x', 'y', 'z']

#Remove 'Letter' column, as the index values now hokd that information
GroupByLetter = make_letters_index.drop('Letter', axis = 1)

GroupByLetter = GroupByLetter.transpose()


SummaryStatsByLetter = (GroupByLetter.describe()).round(decimals = 2)

SummaryStatsByLetter

                                                            Boxplots

In [ ]:
px.box(merged_data).show()

px.box(GroupByLetter).show()

                                                             Bar Graphs

In [ ]:
%matplotlib inline

def BarGraph(df, Title, Color, Y):
    bar_graph = go.Figure(data = [go.Bar(x = df["Letter"], y = df[Y])])
    bar_graph.update_traces(marker_color = Color)
    bar_graph.update_layout(title_text = Title)
    bar_graph.show()

BarGraph(EssayFrequency_df, "Frequencies of Letters in Essay", "darkviolet", 'Essay Frequency')

BarGraph(ResumeFrequency_df, "Frequencies of Letters in Resume", "deeppink", 'Resume Frequency')

BarGraph(LabFrequency_df, "Frequencies of Letters in Lab Report", "deepskyblue", 'Lab Frequency')


In [ ]:
%matplotlib inline

sns.set_palette("hls", 26)

def histogram(df, Title, X):
    hist = sns.displot(df, x = X, bins = 15, hue = 'Letter', multiple = 'stack', stat = 'probability')
    hist.fig.suptitle(Title)
    
histogram(EssayFrequency_df, "Frequencies of Letters in Essay", 'Essay Frequency')
histogram(ResumeFrequency_df, "Frequencies of Letters in Resume", 'Resume Frequency')
histogram(LabFrequency_df, "Frequencies of Letters in Lab Report", 'Lab Frequency')



In [ ]:

def CreateWordCloud(text):
    #Primary Source for writing this function: https://www.geeksforgeeks.org/generating-word-cloud-python/ 
    
    stopwords = set(STOPWORDS)

    # create wordcloud object
    wordcloud = WordCloud(background_color="white", max_words = 200, stopwords = stopwords)
    wordcloud.generate(text)

    plt.figure(figsize = (10, 10), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.show()
    
CreateWordCloud(EssayText)
CreateWordCloud(ResumeText)
CreateWordCloud(LabText)
